# 文本预处理
:label:`sec_text_preprocessing`

我们进行了审查和评估
统计工具
和预测挑战
对于序列数据。
这些数据可以采取多种形式。
明确地
我们将重点关注
在这本书的许多章节中，
文本是最流行的序列数据示例之一。
例如
一篇文章可以简单地看作是一系列单词，甚至是一系列字符。
为了方便我们将来的实验
使用序列数据，
我们将专门介绍这一节
解释文本的常见预处理步骤。
通常，这些步骤是：

1. 将文本作为字符串加载到内存中。
2. 将字符串拆分为标记（例如，单词和字符）。
3. 建立词汇表，将分割标记映射到数字索引。
4. 将文本转换为数字索引序列，以便模型轻松操作。


In [ ]:
%load ../utils/djl-imports

## 读取数据集

首先，我们从H.G.Wells的[*时间机器*]加载文本(http://www.gutenberg.org/ebooks/35).
这是一个只有30000多个单词的相当小的语料库，但是为了我们想要说明的目的，这是很好的。
更现实的文档集合包含数十亿字。
下面的函数将数据集读入文本行列表，其中每一行都是一个字符串。
为了简单起见，这里我们忽略了标点符号和大写字母。


In [ ]:
public String[] readTimeMachine() throws IOException {
    URL url = new URL("http://d2l-data.s3-accelerate.amazonaws.com/timemachine.txt");
    String[] lines;
    try (BufferedReader in = new BufferedReader(new InputStreamReader(url.openStream()))) {
        lines = in.lines().toArray(String[]::new);
    }

    for (int i = 0; i < lines.length; i++) {
        lines[i] = lines[i].replaceAll("[^A-Za-z]+", " ").strip().toLowerCase();
    }
    return lines;
}

String[] lines = readTimeMachine();
System.out.println("# text lines: " + lines.length);
System.out.println(lines[0]);
System.out.println(lines[10]);

## 标记化

下面的`tokenize` 函数
将数组（`lines`）作为输入，
其中，每个元素都是一个文本序列（例如，文本行）。
每个文本序列被拆分为一个标记列表。
*标记*是文本中的基本单位。
最后
返回token列表的列表，
其中每个标记都是一个字符串。


In [ ]:
public String[][] tokenize(String[] lines, String token) throws Exception {
    // 将文本行拆分为单词或字符标记
    String[][] output = new String[lines.length][];
    if (token == "word") {
        for (int i = 0; i < output.length; i++) {
            output[i] = lines[i].split(" ");
        }
    } else if (token == "char") {
        for (int i = 0; i < output.length; i++) {
            output[i] = lines[i].split("");
        }
    } else {
        throw new Exception("ERROR: unknown token type: " + token);
    }
    return output; 
}
String[][] tokens = tokenize(lines, "word");
for (int i = 0; i < 11; i++) {
    System.out.println(Arrays.toString(tokens[i]));
}

## 词汇

token的字符串类型不便于模型使用，因为模型需要数字输入。
现在，让我们构建一个字典（HashMap），通常也被称为*词汇*，将字符串标记映射到从0开始的数字索引中。
为此，我们首先统计训练集中所有文档中的唯一标记，
即 *语料库*，
然后根据每个唯一标记的频率为其分配一个数字索引。
很少出现的标记通常会被移除以降低复杂性。
语料库中不存在或已删除的任何标记都映射到一个特殊的未知标记“&lt;unk&gt;”。
我们可以选择添加保留令牌的列表，例如
“&lt;pad&gt;” 对于填充，
“&lt;bos&gt;” 显示序列的开头，以及“&lt;eos&gt;”用于序列的结尾。


In [ ]:
public class Vocab {
    public int unk;
    public List<Map.Entry<String, Integer>> tokenFreqs;
    public List<String> idxToToken;
    public HashMap<String, Integer> tokenToIdx;

    public Vocab(String[][] tokens, int minFreq, String[] reservedTokens) {
        // 按频率排序
        LinkedHashMap<String, Integer> counter = countCorpus2D(tokens);
        this.tokenFreqs = new ArrayList<Map.Entry<String, Integer>>(counter.entrySet()); 
        Collections.sort(tokenFreqs, 
            new Comparator<Map.Entry<String, Integer>>() { 
                public int compare(Map.Entry<String, Integer> o1, Map.Entry<String, Integer> o2) { 
                    return (o2.getValue()).compareTo(o1.getValue()); 
                }
            });
        
        // 未知标记的索引为0
        this.unk = 0;
        List<String> uniqTokens = new ArrayList<>();
        uniqTokens.add("<unk>");
        Collections.addAll(uniqTokens, reservedTokens);
        for (Map.Entry<String, Integer> entry : tokenFreqs) {
            if (entry.getValue() >= minFreq && !uniqTokens.contains(entry.getKey())) {
                uniqTokens.add(entry.getKey());
            }
        }
        
        this.idxToToken = new ArrayList<>();
        this.tokenToIdx = new HashMap<>();
        for (String token : uniqTokens) {
            this.idxToToken.add(token);
            this.tokenToIdx.put(token, this.idxToToken.size()-1);
        }
    }
    
    public int length() {
        return this.idxToToken.size();
    }
    
    public Integer[] getIdxs(String[] tokens) {
        List<Integer> idxs = new ArrayList<>();
        for (String token : tokens) {
            idxs.add(getIdx(token));
        }
        return idxs.toArray(new Integer[0]);
        
    }
    
    public Integer getIdx(String token) {
        return this.tokenToIdx.getOrDefault(token, this.unk);
    }
    
    
}

public LinkedHashMap<String, Integer> countCorpus(String[] tokens) {
    /* 计算token频率 */
    LinkedHashMap<String, Integer> counter = new LinkedHashMap<>();
    if (tokens.length != 0) {
        for (String token : tokens) {
            counter.put(token, counter.getOrDefault(token, 0)+1);
        }
    }
    return counter;
}

public LinkedHashMap<String, Integer> countCorpus2D(String[][] tokens) {
    /* 将token列表展平为token列表*/
    List<String> allTokens = new ArrayList<String>();
    for (int i = 0; i < tokens.length; i++) {
        for (int j = 0; j < tokens[i].length; j++) {
             if (tokens[i][j] != "") {
                allTokens.add(tokens[i][j]);
             }
        }
    }
    return countCorpus(allTokens.toArray(new String[0]));
}

我们使用时间机器数据集作为语料库构建了一个词汇表。
然后，我们打印前几个频繁标记及其索引。

In [ ]:
Vocab vocab = new Vocab(tokens, 0, new String[0]);
for (int i = 0; i < 10; i++) {
    String token = vocab.idxToToken.get(i);
    System.out.print("(" + token + ", " + vocab.tokenToIdx.get(token) + ") ");
}

现在我们可以将每一行文本转换为一个数字索引列表。


In [ ]:
for (int i : new int[] {0,10}) {
    System.out.println("Words:" + Arrays.toString(tokens[i]));
    System.out.println("Indices:" + Arrays.toString(vocab.getIdxs(tokens[i])));
}

## 把所有的东西放在一起

使用上述函数，我们将所有内容打包到`loadCorpusTimeMachine`函数中，
该函数返回`corpus`，一个标记索引列表，以及`vocab`，时间机器语料库的词汇表。
我们在这里做的修改是：
一） 我们将文本标记为字符，而不是单词，以简化后面章节中的训练；
二）`corpus` 是一个单一的列表，而不是标记列表列表，因为时间机器数据集中的每一行文本不一定是一个句子或段落。


In [ ]:
public Pair<List<Integer>, Vocab> loadCorpusTimeMachine(int maxTokens) throws IOException, Exception {
    /* 返回时间机器数据集的令牌索引和词汇表。 */
    String[] lines = readTimeMachine();
    String[][] tokens = tokenize(lines, "char");
    Vocab vocab = new Vocab(tokens, 0, new String[0]);
    // 因为时间机器数据集中的每个文本行不一定是
    // 句子或段落，将所有文本行展平为一个列表
    List<Integer> corpus = new ArrayList<>();
    for (int i = 0; i < tokens.length; i++) {
        for (int j = 0; j < tokens[i].length; j++) {
            if (tokens[i][j] != "") {
                corpus.add(vocab.getIdx(tokens[i][j]));
            }
        }
    }
    if (maxTokens > 0) {
        corpus = corpus.subList(0, maxTokens);
    }
    return new Pair(corpus, vocab);
}

Pair<List<Integer>, Vocab> corpusVocabPair = loadCorpusTimeMachine(-1);
List<Integer> corpus = corpusVocabPair.getKey();
Vocab vocab = corpusVocabPair.getValue();

System.out.println(corpus.size());
System.out.println(vocab.length());

## 总结

* 文本是序列数据的一种重要形式。
* 为了预处理文本，我们通常将文本拆分为标记，构建词汇表将标记字符串映射为数字索引，并将文本数据转换为标记索引，以便模型进行操作。


## 练习

1. 标记化是一个关键的预处理步骤。它因语言而异。尝试找到另外三种常用的文本标记方法。
2. 在本节的实验中，将文本标记为单词，并改变`Vocab`实例的 `minFreq` 参数。这是如何影响词汇量的？